In [1]:
%reload_ext autoreload
%autoreload 2

In [59]:
import os.path as op
import os
import hcp
import glob
import shutil
from scipy.io import loadmat, savemat

In [3]:
subject = '105923'
base_data_path = op.expanduser('/Users/dengemann/mne-hcp-data')
testing_data_path = op.join(base_data_path, 'mne-hcp-testing')
hcp_data_path = op.join(base_data_path, 'HCP')
data_types = ['task_working_memory', 'task_motor', 'task_story_math', 'rest',
              'noise_subject', 'noise_empty_room']
test_zip = op.join(base_data_path, 'hcp105923')

# copy over edited raw files

In [4]:
def edit(fname):
    fname = (fname.replace(test_zip, testing_data_path)
                  .replace('unprocd', '%s/unprocessed/MEG' % subject)
                  .replace(',ee', ''))
    return fname

In [5]:
fnames = list()
fnames.extend(glob.glob(test_zip + "/*/*/*/config"))
fnames.extend(glob.glob(test_zip + "/*/*/*/*,ee"))
fnames_to = [edit(f) for f in fnames]
new_paths = {op.dirname(f) for f in fnames_to}
for path in new_paths:
    if not op.exists(path):
        os.makedirs(path)

In [6]:
for old_name, new_name in zip(fnames, fnames_to):
    shutil.copy(old_name, new_name)

# copy over files from download 

In [7]:
files_from_subject = hcp.file_mapping.get_s3_keys_meg(
    subject=subject, data_types=data_types,
    outputs=('epochs', 'ica', 'bads', 'trial_info'), run_inds=list(range(3)),
    hcp_path_bucket=hcp_data_path)
files_from_subject.extend(glob.glob(hcp_data_path + '/%s/T1w/*/*/*' % subject))
files_from_subject.extend(hcp.file_mapping.get_s3_keys_anatomy(
    subject=subject,
    freesurfer_outputs=(),
    hcp_path_bucket=hcp_data_path))

In [8]:
files_to_testing = [ff.replace(hcp_data_path, testing_data_path) for ff in files_from_subject]

In [65]:
new_paths = {op.dirname(f) for f in files_to_testing}
for path in new_paths:
    if not op.exists(path):
        os.makedirs(path)

In [67]:
for old_name, new_name in zip(files_from_subject,
                              files_to_testing):
    shutil.copy(old_name, new_name)

# edit epochs files

In [39]:
epochs_files = list()
for data_type in data_types:
    if 'noise' in data_type:
        continue
    n_runs = len(hcp.io.file_mapping.file_mapping.run_map[data_type])
    for run_index in range(n_runs): 
        epochs_files.extend(
            [hcp.file_mapping.get_file_paths(
                subject=subject, hcp_path=testing_data_path,
                output='epochs', data_type=data_type, onset=onset, run_index=run_index)[0]
             for onset in ('stim', 'resp')])

In [43]:
this_epochs_files = [ff for ff in
                     glob.glob(op.join(testing_data_path, '105923/MEG/*/*megpreproc/*mat'))
                     if ff in epochs_files]

In [64]:
mne.epochs._check_decim??

In [52]:
def decim_hcp_mat_epochs(fname, decim=100):
    mat = loadmat(fname, squeeze_me=False)
    data = mat['data']
    for ii, this_times in enumerate(data['time'][0][0][0]):
        data['time'][0][0][0][ii] = this_times[:, ::decim]
    for ii, this_trial in enumerate(data['trial'][0][0][0]):
        data['trial'][0][0][0][ii] = this_trial[:, ::decim]
    data['fsample'][0][0][0][0] /= float(decim)
    mat['data'] = data
    savemat(fname, mat)


In [53]:
for fname in this_epochs_files:
    decim_hcp_mat_epochs(fname)

In [68]:
fname = this_epochs_files[-3]

In [69]:
mat = loadmat(fname, squeeze_me=False)

In [71]:
data['fsample'][0][0][0][0]/float(100)

5.0862752490487884

In [77]:
times = data['time'][0][0][0][0][:, ::100]

In [78]:
times

array([[-1.49962391, -1.30301639, -1.10640886, -0.90980133, -0.71319381,
        -0.51658628, -0.31997875, -0.12337122,  0.0732363 ,  0.26984383,
         0.46645136,  0.66305889,  0.85966641,  1.05627394,  1.25288147,
         1.449489  ,  1.64609652,  1.84270405,  2.03931158,  2.2359191 ,
         2.43252663,  2.62913416,  2.82574169,  3.02234921,  3.21895674,
         3.41556427,  3.6121718 ,  3.80877932]])

In [80]:
decim = 100

In [81]:
for ii, this_trial in enumerate(data['trial'][0][0][0]):
        data['trial'][0][0][0][ii] = this_trial[:, ::decim]

In [163]:
shape = (195,) + data['trial'][0][0][0][0].shape

In [164]:
tmin = times.min()

In [179]:
sfreq = round(data['fsample'][0][0][0][0], 3) / float(decim)

In [166]:
tmax = (shape[2] - 1) / sfreq + tmin

In [168]:
tmax == times.max()

True

In [182]:
tmin == times.min()

False

In [189]:
sfreq = data['fsample'][0][0][0][0] / float(decim)

In [190]:
start_idx = int(round(-1.5 * sfreq))

In [193]:
np.arange(start_idx,
          int(round(tmax * sfreq))) / float(sfreq)

array([-1.57286022, -1.37625269, -1.17964516, -0.98303764, -0.78643011,
       -0.58982258, -0.39321505, -0.19660753,  0.        ,  0.19660753,
        0.39321505,  0.58982258,  0.78643011,  0.98303764,  1.17964516,
        1.37625269,  1.57286022,  1.76946775,  1.96607527,  2.1626828 ,
        2.35929033,  2.55589786,  2.75250538,  2.94911291,  3.14572044,
        3.34232796,  3.53893549])

In [194]:
times

array([[-1.49962391, -1.30301639, -1.10640886, -0.90980133, -0.71319381,
        -0.51658628, -0.31997875, -0.12337122,  0.0732363 ,  0.26984383,
         0.46645136,  0.66305889,  0.85966641,  1.05627394,  1.25288147,
         1.449489  ,  1.64609652,  1.84270405,  2.03931158,  2.2359191 ,
         2.43252663,  2.62913416,  2.82574169,  3.02234921,  3.21895674,
         3.41556427,  3.6121718 ,  3.80877932]])

In [126]:
tmax

3.8087793230661191

In [127]:
tmin

-1.499623914656695

# take care of ICA

In [54]:
ica_fnames = glob.glob(op.join(testing_data_path, '105923/MEG/*/*ica*/*mat'))

In [57]:
def decim_hcp_mat_ica(fname):
    mat = loadmat(fname, squeeze_me=False)
    data = mat['comp_class']
    data['trial'] = 0
    mat['comp_class'] = data
    mat['options'] = []
    savemat(fname, mat)

In [56]:
for fname in ica_fnames:
    decim_hcp_mat_ica(fname)

In [61]:
import mne

In [62]:
mne.EpochsArray?